In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle

raw_csv_data = np.loadtxt('Audiobooks_data (2).csv', delimiter=',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]
import tensorflow as tf

In [ ]:
#Balanced dataset

num_one_targets = int(np.sum(targets_all))

zero_targets_counter = 0

indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [ ]:
#Standardize

scaler_deep_learning = StandardScaler()
scaled_inputs = scaler_deep_learning.fit_transform(unscaled_inputs_equal_priors)

In [ ]:
#Shuffle
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [ ]:

samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1792.0 3579 0.5006985191394244
227.0 447 0.5078299776286354
218.0 448 0.48660714285714285


In [ ]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [ ]:
pickle.dump(scaler_deep_learning, open('scaler_deep_learning.pickle', 'wb'))

In [ ]:
npz = np.load('Audiobooks_data_train.npz')

In [ ]:
train_inputs = npz['inputs'].astype(np.float)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(np.int)

# we load the validation data in the temporary variable
npz = np.load('Audiobooks_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

In [ ]:
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


### Choose the optimizer and the loss function

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 100

# set a maximum number of training epochs
max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )

Epoch 1/100
36/36 - 1s - loss: 0.5400 - accuracy: 0.8055 - val_loss: 0.4144 - val_accuracy: 0.8680
Epoch 2/100
36/36 - 0s - loss: 0.3663 - accuracy: 0.8748 - val_loss: 0.3237 - val_accuracy: 0.8792
Epoch 3/100
36/36 - 0s - loss: 0.3147 - accuracy: 0.8852 - val_loss: 0.3015 - val_accuracy: 0.8814
Epoch 4/100
36/36 - 0s - loss: 0.2962 - accuracy: 0.8919 - val_loss: 0.2856 - val_accuracy: 0.8837
Epoch 5/100
36/36 - 0s - loss: 0.2820 - accuracy: 0.8958 - val_loss: 0.2793 - val_accuracy: 0.8926
Epoch 6/100
36/36 - 0s - loss: 0.2722 - accuracy: 0.8986 - val_loss: 0.2768 - val_accuracy: 0.8926
Epoch 7/100
36/36 - 0s - loss: 0.2654 - accuracy: 0.9008 - val_loss: 0.2687 - val_accuracy: 0.8926
Epoch 8/100
36/36 - 0s - loss: 0.2613 - accuracy: 0.9042 - val_loss: 0.2789 - val_accuracy: 0.8971
Epoch 9/100
36/36 - 0s - loss: 0.2561 - accuracy: 0.9042 - val_loss: 0.2649 - val_accuracy: 0.8926
Epoch 10/100
36/36 - 0s - loss: 0.2521 - accuracy: 0.9067 - val_loss: 0.2692 - val_accuracy: 0.8926
Epoch 11/

In [ ]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.2340 - accuracy: 0.9107


In [ ]:
#Probability of customer to convert
model.predict(test_inputs)#.round(2)

array([[7.96340942e-01, 2.03658968e-01],
       [3.75778443e-04, 9.99624252e-01],
       [9.17137086e-01, 8.28629509e-02],
       [1.70861080e-01, 8.29138875e-01],
       [2.21308604e-01, 7.78691411e-01],
       [9.99806225e-01, 1.93779953e-04],
       [8.92164528e-01, 1.07835464e-01],
       [9.30622637e-01, 6.93773776e-02],
       [9.93618369e-01, 6.38162950e-03],
       [2.20519900e-01, 7.79480100e-01],
       [2.97111243e-01, 7.02888787e-01],
       [8.08885455e-01, 1.91114604e-01],
       [3.03354958e-04, 9.99696732e-01],
       [1.79054434e-04, 9.99821007e-01],
       [2.10346386e-01, 7.89653599e-01],
       [3.13745055e-04, 9.99686241e-01],
       [2.63629884e-01, 7.36370087e-01],
       [9.02340233e-01, 9.76596996e-02],
       [9.03183520e-01, 9.68164802e-02],
       [2.52462447e-01, 7.47537553e-01],
       [1.00000000e+00, 2.60145529e-13],
       [3.71611655e-01, 6.28388286e-01],
       [9.90846574e-01, 9.15350392e-03],
       [9.25825655e-01, 7.41743445e-02],
       [1.000000

In [ ]:
np.argmax(model.predict(test_inputs),axis=1)

array([0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,

In [ ]:
model.save('audiobooks_model.h5')

In [16]:
scaler_deep_learning = pickle.load(open('scaler_deep_learning.pickle', 'rb'))
# To load the model, we use the TensorFlow (Keras) function relevant for the operation
model = tf.keras.models.load_model('audiobooks_model.h5')